In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
client = MongoClient('localhost', 27017)
db = client['aoe2']
coll = db['games']

In [3]:
driver = webdriver.Remote(
    command_executor='http://localhost:4444/wd/hub',
    desired_capabilities={
        'browserName':'firefox',
        'javascriptEnabled': True
    }
)
url = 'https://aoe2.net/#aoe2de-matches-ongoing'

In [20]:
driver.get(url)
sleep(2)
soup = bs(driver.page_source, 'html.parser')

In [31]:
def parse_player_data(cells):
    player_data = cells[2].contents
    players = []
    for player in player_data[1].find_all('td', class_='player-color'):
        civ = player.next_sibling
        rating = civ.next_sibling
        player_name = rating.next_sibling
        players.append({
            'civ':civ.img['title'],
            'rating':int(rating.text) if rating.text else '',
            'Player_name': player_name.text,
        })
    return players

In [32]:
def parse_game_data(cells):
    game_data = {
        'leaderboard': cells[0].contents[0],
        'link': cells[0].a['href'],
        'AVG rating': cells[1].text,
        'Map':cells[3].text,
        'Server':cells[4].text,
        'Game Age':cells[5].text,
        'Players':parse_player_data(cells)
    }
    return game_data

In [36]:
def games_from_page(page):
    games_table = page.find('table', id='aoe2de-matches-table')
    game_list = []
    for row in games_table.tbody.children:
        cells = row.contents
        game_list.append(parse_game_data(cells))
    return game_list
    

In [40]:
coll.insert_many(games_from_page(soup))